In [ ]:
#pip install pandas
#pip install numpy
#pip install -U scikit-learn

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Counter proporciona un diccionario donde cada elemento distinto se almacena 
# como claves con un valor asociado igual a su cuenta
from collections import Counter
 

import sklearn
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
 
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
 


In [ ]:

# Cargar los datos y separar variables y etiqueta
df = pd.read_csv('datos/datos10.csv')
print(df.info())
print(df.head())

X = df.drop('clase_objetivo', axis = 1)
y = df['clase_objetivo']

# Dividir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definir las variables que requieren preprocesamiento
numeric_features = ['variable1', 'variable2']
categorical_features = ['variable_categorica']

# Crear un pipeline de preprocesamiento
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Incluir el clasificador en el pipeline
model = KNeighborsClassifier()

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# Definir el grid con los parámetros
param_grid = {
    'classifier__n_neighbors': [3, 7, 11],
    'classifier__p': [1, 2]
}

# Crear un obejeto GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Entrenar el grid en el conjunto de datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros:", grid_search.best_params_)

# Acceder al mejor modelo del grid_search
best_model = grid_search.best_estimator_
print(best_model) # esto es un pipeline incluye el preprocesado

# Utilizar el mejor modelo para hacer las precciones en el conjunto de test
y_pred = best_model.predict(X_test)

# Evaluar la accuracy en el conjunto de test
accuracy_test = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy_test)